<font size="5">**Project Summary**</font><br>
In this competition, the main task is to do surface time series classification. 1d convolution is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The whole code is written in Pytorch.

<font size="5">**External libs we are using**</font><br>

In [72]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Data preparations**</font><br>

In this project, I use the raw data as input of the network. I concatenated all datasets into one single numpy array. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [73]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data_arr, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data_arr
    sz = train_size

    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("input one dimension shape")
    print(raw[0].shape)
    print("training data length")
    print(len(trn_idx))
    print("validation data le")
    print(len(val_idx))
    print("testing d")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[0][:sz][trn_idx]).float(),
        torch.tensor(raw[1][:sz][trn_idx]).float(),
        torch.tensor(raw[2][:sz][trn_idx]).float(),
        torch.tensor(raw[3][:sz][trn_idx]).float(),
        torch.tensor(raw[4][:sz][trn_idx]).float(),
        torch.tensor(raw[5][:sz][trn_idx]).float(),
        torch.tensor(raw[6][:sz][trn_idx]).float(),
        torch.tensor(raw[7][:sz][trn_idx]).float(),
        torch.tensor(raw[8][:sz][trn_idx]).float(),
        torch.tensor(target[:sz][trn_idx]).long())
    
    val_ds = TensorDataset(
        torch.tensor(raw[0][:sz][val_idx]).float(),
        torch.tensor(raw[1][:sz][val_idx]).float(), 
        torch.tensor(raw[2][:sz][val_idx]).float(), 
        torch.tensor(raw[3][:sz][val_idx]).float(), 
        torch.tensor(raw[4][:sz][val_idx]).float(), 
        torch.tensor(raw[5][:sz][val_idx]).float(), 
        torch.tensor(raw[6][:sz][val_idx]).float(), 
        torch.tensor(raw[7][:sz][val_idx]).float(), 
        torch.tensor(raw[8][:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    
    tst_ds = TensorDataset(
        torch.tensor(raw[0][:sz][tst_idx]).float(),
        torch.tensor(raw[1][:sz][tst_idx]).float(),
        torch.tensor(raw[2][:sz][tst_idx]).float(),
        torch.tensor(raw[3][:sz][tst_idx]).float(),
        torch.tensor(raw[4][:sz][tst_idx]).float(),
        torch.tensor(raw[5][:sz][tst_idx]).float(),
        torch.tensor(raw[6][:sz][tst_idx]).float(),
        torch.tensor(raw[7][:sz][tst_idx]).float(),
        torch.tensor(raw[8][:sz][tst_idx]).float(),
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [74]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [75]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d_channel_0 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
        )
            
            
        
        self.conv1d_channel_1 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_2 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_3 = nn.Sequential(
             nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_4 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_5 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
        
        self.conv1d_channel_6 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
           
        )
        
        self.conv1d_channel_7 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )
            

       
        
        self.conv1d_channel_8 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            nn.Conv1d(16, 24, 8, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.Conv1d(24, 32, 5, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
        )

            

        self.dense = nn.Sequential(
            nn.Linear(288, 84),  nn.ReLU(),nn.Dropout(),
            nn.Linear(84, 36),  nn.ReLU(),nn.Dropout(),
            nn.Linear(36, no),  nn.ReLU()
        )
        
            

    def forward(self, t_channel_0, t_channel_1, t_channel_2, t_channel_3, t_channel_4, t_channel_5, t_channel_6, t_channel_7, t_channel_8):
        conv1d_out_channel_0 = self.conv1d_channel_0(t_channel_0)
        conv1d_out_channel_1 = self.conv1d_channel_1(t_channel_1)
        conv1d_out_channel_2 = self.conv1d_channel_2(t_channel_2)
        conv1d_out_channel_3 = self.conv1d_channel_3(t_channel_3)
        conv1d_out_channel_4 = self.conv1d_channel_4(t_channel_4)
        conv1d_out_channel_5 = self.conv1d_channel_5(t_channel_5)
        conv1d_out_channel_6 = self.conv1d_channel_6(t_channel_6)
        conv1d_out_channel_7 = self.conv1d_channel_7(t_channel_7)
        conv1d_out_channel_8 = self.conv1d_channel_8(t_channel_8)
        
        t_in = torch.cat([conv1d_out_channel_0,conv1d_out_channel_1, conv1d_out_channel_2, conv1d_out_channel_3, conv1d_out_channel_4, conv1d_out_channel_5, conv1d_out_channel_6, conv1d_out_channel_7, conv1d_out_channel_8], dim=1)
        res = t_in.view(t_in.size(0), -1)
        out = self.dense(res)
        return out
        

<font size="5">**Initial setup for random seed and project path**</font><br>

In [76]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

raw_dim_data = [None]*9
#print(raw_dim_data)
for i in range(0, 9):
    raw_dim_data[i] = raw_arr[:,i,:]
#    print("raw data shape")
    
    raw_dim_data[i] = raw_dim_data[i].reshape([7626,1,128])
#    print(raw_dim_data[i].shape)
    
# print("raw array shape")
# print(raw_arr.shape)
# print("label array shape")
# print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_dim_data), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

total data length
3810
input one dimension shape
(7626, 1, 128)
training data length
2286
validation data le
762
testing d
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [ ]:
raw_feat = raw_arr.shape[1]


lr = 0.002
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        #x_raw, y_batch = [t.to(device) for t in batch]
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        
#         print("channel 0 type")
#         print(type(x_channel_0))
#         print("channel 0 shape")
#         print(x_channel_0.shape)
#         print("batch type")
#         print(type(batch))
#         print(len(batch))
#         print(batch[0].shape)
#         print(batch[9].shape)

        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        
        
#         out = []
#         with torch.no_grad():
#             for x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch in batch:
#                 output = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
#                 out.append(output.detach())
#         out = torch.cat(out)
        

    
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    
    for batch in val_dl:
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.3139. Acc.: 21.78%
Epoch:   1. Loss: 1.3139. Acc.: 21.78%
Epoch 1 best model saved with accuracy: 21.78%
Epoch:   2. Loss: 1.2944. Acc.: 21.78%
Epoch:   2. Loss: 1.2944. Acc.: 21.78%
Epoch:   3. Loss: 1.2618. Acc.: 21.78%
Epoch:   4. Loss: 1.2461. Acc.: 28.48%
Epoch:   4. Loss: 1.2461. Acc.: 28.48%
Epoch 4 best model saved with accuracy: 28.48%
Epoch:   5. Loss: 1.2251. Acc.: 38.71%
Epoch 5 best model saved with accuracy: 38.71%
Epoch:   6. Loss: 1.2085. Acc.: 37.40%
Epoch:   7. Loss: 1.1964. Acc.: 36.09%
Epoch:   8. Loss: 1.1653. Acc.: 33.86%
Epoch:   8. Loss: 1.1653. Acc.: 33.86%
Epoch:   9. Loss: 1.1543. Acc.: 35.70%
Epoch:  10. Loss: 1.1343. Acc.: 36.09%
Epoch:  11. Loss: 1.1247. Acc.: 33.86%
Epoch:  12. Loss: 1.1141. Acc.: 33.99%
Epoch:  13. Loss: 1.1176. Acc.: 37.80%
Epoch:  14. Loss: 1.1067. Acc.: 32.55%
Epoch:  15. Loss: 1.1036. Acc.: 37.27%
Epoch:  16. Loss: 1.0960. Acc.: 36.75%
Epoch:  16. Loss: 1.0960. Acc.: 36.75%
Epoch:  17. Loss: 1

Epoch: 176. Loss: 0.7232. Acc.: 58.40%
Epoch: 177. Loss: 0.7333. Acc.: 53.81%
Epoch: 178. Loss: 0.7272. Acc.: 58.66%
Epoch: 179. Loss: 0.7180. Acc.: 52.23%
Epoch: 180. Loss: 0.7341. Acc.: 59.45%
Epoch 180 best model saved with accuracy: 59.45%
Epoch: 181. Loss: 0.7243. Acc.: 55.25%
Epoch: 182. Loss: 0.7431. Acc.: 58.53%
Epoch: 183. Loss: 0.7247. Acc.: 53.28%
Epoch: 184. Loss: 0.7121. Acc.: 55.25%
Epoch: 185. Loss: 0.7129. Acc.: 54.33%
Epoch: 186. Loss: 0.7118. Acc.: 58.01%
Epoch: 187. Loss: 0.7187. Acc.: 54.46%
Epoch: 188. Loss: 0.7178. Acc.: 58.14%
Epoch: 189. Loss: 0.7108. Acc.: 57.35%
Epoch: 190. Loss: 0.7087. Acc.: 55.12%
Epoch: 191. Loss: 0.7031. Acc.: 58.01%
Epoch: 192. Loss: 0.7411. Acc.: 57.48%
Epoch: 193. Loss: 0.7244. Acc.: 54.72%
Epoch: 194. Loss: 0.7147. Acc.: 55.77%
Epoch: 195. Loss: 0.7139. Acc.: 55.51%
Epoch: 196. Loss: 0.7038. Acc.: 56.17%
Epoch: 197. Loss: 0.7274. Acc.: 57.61%
Epoch: 198. Loss: 0.7211. Acc.: 57.09%
Epoch: 199. Loss: 0.7219. Acc.: 57.09%
Epoch: 200. Los

Epoch: 378. Loss: 0.6231. Acc.: 59.06%
Epoch: 379. Loss: 0.6305. Acc.: 57.22%
Epoch: 380. Loss: 0.6333. Acc.: 59.58%
Epoch: 381. Loss: 0.6406. Acc.: 57.35%
Epoch: 382. Loss: 0.6432. Acc.: 57.09%
Epoch: 383. Loss: 0.6246. Acc.: 58.14%
Epoch: 384. Loss: 0.6311. Acc.: 58.40%
Epoch: 385. Loss: 0.6389. Acc.: 59.45%
Epoch: 386. Loss: 0.6440. Acc.: 55.91%
Epoch: 387. Loss: 0.6495. Acc.: 59.97%
Epoch: 388. Loss: 0.6378. Acc.: 56.04%
Epoch: 389. Loss: 0.6361. Acc.: 57.61%
Epoch: 390. Loss: 0.6286. Acc.: 54.20%
Epoch: 391. Loss: 0.6245. Acc.: 58.79%
Epoch: 392. Loss: 0.6213. Acc.: 56.56%
Epoch: 393. Loss: 0.6390. Acc.: 59.58%
Epoch: 394. Loss: 0.6327. Acc.: 56.43%
Epoch: 395. Loss: 0.6267. Acc.: 58.66%
Epoch: 396. Loss: 0.6370. Acc.: 52.49%
Epoch: 397. Loss: 0.6296. Acc.: 58.01%
Epoch: 398. Loss: 0.6286. Acc.: 55.64%
Epoch: 399. Loss: 0.6332. Acc.: 59.06%
Epoch: 400. Loss: 0.6281. Acc.: 59.45%
Epoch: 401. Loss: 0.6279. Acc.: 58.79%
Epoch: 402. Loss: 0.6285. Acc.: 58.27%
Epoch: 403. Loss: 0.6190.

Epoch: 585. Loss: 0.6008. Acc.: 59.58%
Epoch: 586. Loss: 0.6014. Acc.: 58.27%
Epoch: 587. Loss: 0.6005. Acc.: 59.45%
Epoch: 588. Loss: 0.5999. Acc.: 61.02%
Epoch: 589. Loss: 0.6076. Acc.: 58.40%
Epoch: 590. Loss: 0.5857. Acc.: 61.55%
Epoch: 591. Loss: 0.5896. Acc.: 56.17%
Epoch: 592. Loss: 0.6127. Acc.: 60.63%
Epoch: 593. Loss: 0.6141. Acc.: 57.74%
Epoch: 594. Loss: 0.5879. Acc.: 59.71%
Epoch: 595. Loss: 0.6089. Acc.: 59.97%
Epoch: 596. Loss: 0.5938. Acc.: 59.58%
Epoch: 597. Loss: 0.5984. Acc.: 58.14%
Epoch: 598. Loss: 0.5878. Acc.: 57.74%
Epoch: 599. Loss: 0.5952. Acc.: 58.40%
Epoch: 600. Loss: 0.5975. Acc.: 56.56%
Epoch: 601. Loss: 0.5961. Acc.: 59.32%
Epoch: 602. Loss: 0.5882. Acc.: 56.43%
Epoch: 603. Loss: 0.5880. Acc.: 58.27%
Epoch: 604. Loss: 0.5992. Acc.: 60.63%
Epoch: 605. Loss: 0.6035. Acc.: 62.34%
Epoch 605 best model saved with accuracy: 62.34%
Epoch: 606. Loss: 0.5770. Acc.: 56.96%
Epoch: 607. Loss: 0.5795. Acc.: 61.15%
Epoch: 608. Loss: 0.5847. Acc.: 57.74%
Epoch: 609. Los

Epoch: 792. Loss: 0.5671. Acc.: 62.99%
Epoch: 793. Loss: 0.5800. Acc.: 62.20%
Epoch: 794. Loss: 0.5975. Acc.: 61.29%
Epoch: 795. Loss: 0.5584. Acc.: 62.34%
Epoch: 796. Loss: 0.5861. Acc.: 61.15%
Epoch: 797. Loss: 0.5969. Acc.: 60.37%
Epoch: 798. Loss: 0.5808. Acc.: 59.71%
Epoch: 799. Loss: 0.5745. Acc.: 62.20%
Epoch: 800. Loss: 0.5832. Acc.: 59.32%
Epoch: 801. Loss: 0.5941. Acc.: 60.10%
Epoch: 802. Loss: 0.5776. Acc.: 60.24%
Epoch: 803. Loss: 0.5789. Acc.: 60.63%
Epoch: 804. Loss: 0.5830. Acc.: 61.81%
Epoch: 805. Loss: 0.5594. Acc.: 61.68%
Epoch: 806. Loss: 0.5712. Acc.: 60.50%
Epoch: 807. Loss: 0.5638. Acc.: 57.87%
Epoch: 808. Loss: 0.5803. Acc.: 61.42%
Epoch: 809. Loss: 0.5701. Acc.: 61.02%
Epoch: 810. Loss: 0.5815. Acc.: 60.24%
Epoch: 811. Loss: 0.5749. Acc.: 61.55%
Epoch: 812. Loss: 0.5763. Acc.: 61.02%
Epoch: 813. Loss: 0.5943. Acc.: 58.40%
Epoch: 814. Loss: 0.5812. Acc.: 61.29%
Epoch: 815. Loss: 0.5771. Acc.: 62.07%
Epoch: 816. Loss: 0.5839. Acc.: 61.42%
Epoch: 817. Loss: 0.5593.

Epoch: 999. Loss: 0.5604. Acc.: 63.52%
Epoch: 1000. Loss: 0.5520. Acc.: 64.70%
Epoch: 1001. Loss: 0.5498. Acc.: 61.81%
Epoch: 1002. Loss: 0.5563. Acc.: 62.34%
Epoch: 1003. Loss: 0.5500. Acc.: 61.94%
Epoch: 1004. Loss: 0.5515. Acc.: 62.47%
Epoch: 1005. Loss: 0.5409. Acc.: 63.52%
Epoch: 1006. Loss: 0.5565. Acc.: 61.94%
Epoch: 1007. Loss: 0.5361. Acc.: 62.34%
Epoch: 1008. Loss: 0.5718. Acc.: 62.07%
Epoch: 1009. Loss: 0.5461. Acc.: 62.99%
Epoch: 1010. Loss: 0.5547. Acc.: 62.60%
Epoch: 1011. Loss: 0.5623. Acc.: 62.86%
Epoch: 1012. Loss: 0.5652. Acc.: 61.15%
Epoch: 1013. Loss: 0.5550. Acc.: 63.25%
Epoch: 1014. Loss: 0.5708. Acc.: 61.02%
Epoch: 1015. Loss: 0.5568. Acc.: 61.68%
Epoch: 1016. Loss: 0.5657. Acc.: 62.07%
Epoch: 1017. Loss: 0.5549. Acc.: 62.20%
Epoch: 1018. Loss: 0.5539. Acc.: 61.42%
Epoch: 1019. Loss: 0.5241. Acc.: 63.78%
Epoch: 1020. Loss: 0.5692. Acc.: 61.29%
Epoch: 1021. Loss: 0.5543. Acc.: 61.55%
Epoch: 1022. Loss: 0.5696. Acc.: 62.60%
Epoch: 1023. Loss: 0.5460. Acc.: 62.60%
E

<font size="5">**Compute result on testing data**</font><br>

In [ ]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))

<font size="5">**Store final result**</font><br>

In [ ]:
'''
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")
'''